In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math

import nibabel as nib
import nilearn.plotting as plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import hcp_utils as hcp


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_recall_curve
from sklearn.pipeline import Pipeline

import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTETomek

from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests
from scipy.stats import pearsonr




open_access_data = pd.read_csv("unrestricted_data.csv")
restricted_data = pd.read_csv("RESTRICTED_BEHAVIORAL_DATA.csv")
subject_data = open_access_data.merge(restricted_data, how = 'inner', on = 'Subject')

file_path = "parcellations"
file_list = os.listdir(file_path)
mmp = {}
ca = {}
for filename in file_list:
    subject_id = filename[4:10]

    if subject_id.isdigit():
        subject_id = int(subject_id)

        if "mmp" in filename:
            mmp_file = np.load(file_path + "/" + filename)
            if mmp_file.shape == (3600, 379):
                mmp[subject_id] = mmp_file
        if "ca" in filename:
            ca_file = np.load(file_path + "/" + filename)
            if ca_file.shape == (3600, 718):
                ca[subject_id] = ca_file

mmp_data = pd.DataFrame({
    'Subject': list(mmp.keys()),
    'Brain_Data': list(mmp.values())  # (3600, 379) arrays
})


mmp_data = subject_data.merge(mmp_data, on='Subject', how='inner')

ca_data = pd.DataFrame({
    'Subject': list(ca.keys()),
    'Brain_Data': list(ca.values())  # (3600, 718) arrays
})


ca_data = subject_data.merge(ca_data, on='Subject', how='inner')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [55]:
mmp_data["Handedness_Cat"] = mmp_data["Handedness"].apply(lambda x: "Left" if x < 0 else "Right")
mmp_data["Gender_Cat"] = (mmp_data["Gender"] == "M").astype("int")
hand_data = mmp_data[["Subject", "Gender_Cat", "Race", "Handedness", "Handedness_Cat", "Gender"]]
brain_region = list(hcp.mmp.labels.values())[1:]

In [56]:
brain_data = np.array(mmp_data["Brain_Data"].tolist())
brain_data_mean = brain_data.mean(axis = 1)

In [58]:
brain_df = pd.DataFrame(brain_data_mean, columns = brain_region)
brain_df = pd.concat([mmp_data["Subject"], mmp_data["Handedness"], brain_df], axis = 1)
brain_df

,Subject,Handedness,L_V1,L_MST,L_V6,L_V2,L_V3,L_V4,L_V8,L_4,...,diencephalon_right,hippocampus_left,hippocampus_right,pallidum_left,pallidum_right,putamen_left,putamen_right,thalamus_left,thalamus_right,brainStem
0,100610,85,4.947637e-05,-0.000043,-0.000045,0.000092,-0.000077,-0.000073,0.000060,0.000021,...,-6.411604e-05,-0.000032,-0.000034,-0.000016,-0.000022,-0.000016,0.000111,-6.862843e-05,-0.000155,-0.000007
1,102311,10,-6.433758e-06,-0.000038,-0.000011,-0.000037,-0.000031,0.000004,0.000004,-0.000078,...,1.328788e-04,-0.000072,0.000034,0.000038,-0.000016,0.000025,-0.000068,-2.328013e-05,-0.000057,0.000070
2,102816,100,2.243658e-05,-0.000020,0.000011,-0.000054,-0.000017,0.000016,-0.000049,-0.000039,...,-1.224528e-05,-0.000016,0.000019,-0.000010,0.000022,0.000025,0.000074,3.714224e-05,0.000027,0.000059
3,104416,65,-2.541886e-05,0.000042,-0.000023,-0.000009,0.000018,-0.000085,-0.000026,-0.000161,...,-8.299854e-05,-0.000076,0.000013,-0.000020,0.000015,0.000035,0.000053,1.392636e-04,0.000013,-0.000020
4,105923,35,5.285111e-07,0.000033,-0.000020,0.000040,0.000083,-0.000012,0.000047,0.000151,...,1.094812e-04,-0.000039,-0.000054,0.000015,0.000070,0.000028,-0.000025,1.101115e-04,-0.000102,0.000100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,943862,95,-3.710022e-05,0.000008,-0.000013,-0.000051,-0.000081,0.000048,0.000045,-0.000092,...,-1.679136e-05,0.000024,-0.000052,0.000020,-0.000014,-0.000052,0.000023,4.366062e-05,-0.000031,-0.000108
168,958976,85,-2.662953e-05,-0.000014,-0.000108,0.000012,-0.000046,0.000035,0.000001,-0.000016,...,-8.206176e-05,0.000001,-0.000034,0.000014,0.000028,-0.000071,0.000096,-5.172395e-05,0.000074,0.000014
169,966975,40,8.370943e-06,0.000025,-0.000002,0.000012,0.000028,0.000017,-0.000005,-0.000001,...,-4.187690e-05,-0.000009,-0.000030,-0.000010,0.000041,-0.000030,-0.000027,-3.609061e-07,-0.000011,-0.000023
170,971160,80,-6.337497e-05,0.000021,0.000008,-0.000063,0.000040,-0.000069,-0.000009,-0.000023,...,-5.404808e-05,0.000012,-0.000023,-0.000003,-0.000020,-0.000056,0.000014,-6.997887e-05,-0.000047,0.000168


In [59]:
corr_matrix = brain_df.corr()
handedness_corr = corr_matrix['Handedness'].drop('Handedness')

In [60]:
handedness_corr

Subject           0.031523
L_V1             -0.003256
L_MST            -0.097909
L_V6             -0.024817
L_V2             -0.024854
                    ...   
putamen_left     -0.085368
putamen_right     0.101065
thalamus_left     0.122002
thalamus_right    0.124893
brainStem        -0.089580
Name: Handedness, Length: 380, dtype: float64

Adjusted p value

In [72]:
correlation_results = {}
for region in brain_region:
    corr_coef, p_value = pearsonr(brain_df['Handedness'], brain_df[region])
    correlation_results[region] = {'correlation': corr_coef, 'p_value': p_value}

results_df = pd.DataFrame.from_dict(correlation_results, orient='index')
print("\nCorrelation coefficients and p-values:\n", results_df)

results_df = pd.DataFrame.from_dict(correlation_results, orient='index')

# Perform multiple comparisons correction (FDR correction using Benjamini/Hochberg)
reject, pvals_corrected, _, _ = multipletests(results_df['p_value'], alpha=0.4, method='fdr_bh')
results_df['p_adjusted'] = pvals_corrected
results_df['significant'] = reject

# Display significant regions
significant_regions = results_df[results_df['significant']]
print("Significant Regions:")
print(significant_regions)

# Optionally, if you want just the list of significant region labels:
significant_region_labels = significant_regions.index.tolist()
print("\nList of Significant Region Labels:")
print(significant_region_labels)

Significant Regions:
         correlation   p_value  p_adjusted  significant
L_1         0.203000  0.007568    0.358520         True
L_TA2      -0.262974  0.000492    0.186326         True
L_TE1a     -0.209284  0.005864    0.317489         True
L_pOFC     -0.209960  0.005703    0.317489         True
R_PEF      -0.240134  0.001509    0.197029         True
R_11l       0.234492  0.001960    0.197029         True
R_OP2-3     0.214241  0.004771    0.317489         True
R_PFcm     -0.233199  0.002079    0.197029         True

List of Significant Region Labels:
['L_1', 'L_TA2', 'L_TE1a', 'L_pOFC', 'R_PEF', 'R_11l', 'R_OP2-3', 'R_PFcm']


Unadjusted p value

In [73]:
# Initialize a dictionary to store correlation coefficients and p-values
correlation_results = {}

# Compute Pearson correlation between handedness and each brain region
for region in brain_region:
    corr_coef, p_value = pearsonr(brain_df['Handedness'], brain_df[region])
    correlation_results[region] = {'correlation': corr_coef, 'p_value': p_value}

# Convert the results to a DataFrame:
results_df = pd.DataFrame.from_dict(correlation_results, orient='index')

# Identify significant regions based on raw (unadjusted) p-values (< 0.05)
alpha = 0.05
significant_regions = results_df[results_df['p_value'] < alpha]

print("Significant Regions (Unadjusted):")
print(significant_regions)

# Optionally, get just the list of significant region labels:
significant_region_labels = significant_regions.index.tolist()
print("\nList of Significant Region Labels (Unadjusted):")
print(significant_region_labels)

Significant Regions (Unadjusted):
         correlation   p_value
L_SFL       0.167511  0.028062
L_1         0.203000  0.007568
L_44        0.177708  0.019689
L_OFC      -0.157328  0.039289
L_TA2      -0.262974  0.000492
L_PHA1      0.153816  0.043950
L_TE1a     -0.209284  0.005864
L_DVT      -0.173941  0.022489
L_pOFC     -0.209960  0.005703
R_PEF      -0.240134  0.001509
R_VIP      -0.178551  0.019106
R_1         0.164633  0.030917
R_11l       0.234492  0.001960
R_OP4      -0.150539  0.048707
R_OP2-3     0.214241  0.004771
R_PFcm     -0.233199  0.002079
R_H         0.160612  0.035315
R_31pd     -0.152818  0.045355

List of Significant Region Labels (Unadjusted):
['L_SFL', 'L_1', 'L_44', 'L_OFC', 'L_TA2', 'L_PHA1', 'L_TE1a', 'L_DVT', 'L_pOFC', 'R_PEF', 'R_VIP', 'R_1', 'R_11l', 'R_OP4', 'R_OP2-3', 'R_PFcm', 'R_H', 'R_31pd']
